In [ ]:
# Standard library imports
import os
import sys

# Contributed library imports
import numpy as np
from pathlib import Path

# Our imports
from multiview_mapping_toolkit.cameras.derived_cameras import MetashapeCameraSet
from multiview_mapping_toolkit.meshes import TexturedPhotogrammetryMesh
from multiview_mapping_toolkit.segmentation import (
    LookUpSegmentor,
    SegmentorPhotogrammetryCameraSet,
)

sys.path.append("../..")
from constants import (
    get_mesh_filename,
    get_camera_filename,
    get_predicted_vector_labels_filename,
    get_numpy_export_faces_texture_filename,
    get_oblique_images_folder,
    get_DTM_filename,
    get_prediction_folder,
    get_image_folder,
    get_IDs_to_labels,
    LABELS_FILENAME,
)

# Set constants
You should be able to define most of the behavior from these constants

In [ ]:
TRAINING_SITES = sorted(["none", "none"])
PREDICTION_SITE_NAME = "none"

In [ ]:
## Parameters to control the outputs
# Repeat the labeling process
RETEXTURE = False
# Points less than this height (meters) above the DTM are considered ground
HEIGHT_ABOVE_GROUND_THRESH = 2
# The image is downsampled to this fraction for accelerated rendering
RENDER_IMAGE_SCALE = 0.25
# Cameras within this distance of the traing data are used in the rendering process
BUFFER_RADIUS_METERS = 50
# Downsample target
DOWNSAMPLE_TARGET = 1
# Image downsample factor for aggregation
AGGREGATE_IMAGE_SCALE = 1 

# Aggregate predictions

In [ ]:
MESH_FILENAME = get_mesh_filename(PREDICTION_SITE_NAME)
CAMERAS_FILENAME = get_camera_filename(PREDICTION_SITE_NAME)
IDs_TO_LABELS = get_IDs_to_labels()

mesh = TexturedPhotogrammetryMesh(
    MESH_FILENAME,
    transform_filename=CAMERAS_FILENAME,
    ROI=LABELS_FILENAME,
    ROI_buffer_meters=BUFFER_RADIUS_METERS,
    IDs_to_labels=IDs_TO_LABELS,
)
print("Done creating mesh")

In [ ]:
IMAGE_FOLDER = get_image_folder(PREDICTION_SITE_NAME)
OBLIQUE_IMAGES_FOLDER = get_oblique_images_folder(PREDICTION_SITE_NAME)

# Create camera set
print(CAMERAS_FILENAME)
camera_set = MetashapeCameraSet(CAMERAS_FILENAME, IMAGE_FOLDER)
# Extract cameras near the training data
training_camera_set = camera_set.get_subset_ROI(
    ROI=LABELS_FILENAME, buffer_radius_meters=BUFFER_RADIUS_METERS
)
training_camera_set = training_camera_set.get_cameras_in_folder(OBLIQUE_IMAGES_FOLDER)
print("About to vis")
# %%
mesh.vis(camera_set=training_camera_set, force_xvfb=True)

In [ ]:
PREDICTIONS_FOLDER = get_prediction_folder(
    prediction_site=PREDICTION_SITE_NAME, training_sites=TRAINING_SITES, is_ortho=False
)
# %%
segmentor = LookUpSegmentor(
    base_folder=IMAGE_FOLDER,
    lookup_folder=PREDICTIONS_FOLDER,
    num_classes=8,
)

segmentor_camera_set = SegmentorPhotogrammetryCameraSet(
    training_camera_set, segmentor=segmentor
)

aggregated_face_labels, _, _ = mesh.aggregate_viewpoints_pytorch3d(
    segmentor_camera_set,
    image_scale=AGGREGATE_IMAGE_SCALE,
)

In [ ]:
DTM_FILE = get_DTM_filename(PREDICTION_SITE_NAME)
NUMPY_EXPORT_FACES_TEXTURE_FILE = get_numpy_export_faces_texture_filename(PREDICTION_SITE_NAME)
PREDICTED_VECTOR_LABELS_FILE = get_predicted_vector_labels_filename(PREDICTION_SITE_NAME)


# %%
predicted_face_classes = np.argmax(
    aggregated_face_labels, axis=1, keepdims=True
).astype(float)
predicted_face_classes = mesh.label_ground_class(
    labels=predicted_face_classes,
    height_above_ground_threshold=HEIGHT_ABOVE_GROUND_THRESH,
    DTM_file=DTM_FILE,
    ground_ID=np.nan,
    set_mesh_texture=False,
)
NUMPY_EXPORT_FACES_TEXTURE_FILE.parent.mkdir(parents=True, exist_ok=True)
np.save(NUMPY_EXPORT_FACES_TEXTURE_FILE, predicted_face_classes)
# %%
mesh.vis(
    vis_scalars=predicted_face_classes,
    force_xvfb=True,
)

mesh.export_face_labels_vector(
    face_labels=np.squeeze(predicted_face_classes),
    export_file=PREDICTED_VECTOR_LABELS_FILE,
    vis=True,
    vis_kwargs={"cmap": "tab10", "vmin": -0.5, "vmax": 9.5},
)